**Cryptocurrency-predicting finance Recurrent Neural Network**

INTRODUCTION

Creating a recurrent neural network to predict against a time-series dataset, which is going to be cryptocurrency prices.
The data we'll be using is Open, High, Low, Close, Volume data for Bitcoin, Ethereum, Litecoin and Bitcoin Cash.

In [1]:
import pandas as pd

df = pd.read_csv('../input/crypto_data/crypto_data/LTC-USD.csv', names=['time', 'low', 'high', 'open', 'close', 'volume'])

print(df.head())

         time        low     ...          close      volume
0  1528968660  96.580002     ...      96.580002    9.647200
1  1528968720  96.449997     ...      96.660004  314.387024
2  1528968780  96.470001     ...      96.570000   77.129799
3  1528968840  96.449997     ...      96.500000    7.216067
4  1528968900  96.279999     ...      96.389999  524.539978

[5 rows x 6 columns]


In [2]:
import pandas as pd
import os
 
main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = f'../input/crypto_data/crypto_data/{ratio}.csv'  # get the full path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  # how did we do??

for c in main_df.columns:
    print(c)

BTC-USD
LTC-USD
BCH-USD
ETH-USD
            BTC-USD_close       ...        ETH-USD_volume
time                            ...                      
1528968720    6487.379883       ...             26.019083
1528968780    6479.410156       ...              8.449400
1528968840    6479.410156       ...             26.994646
1528968900    6479.979980       ...             77.355759
1528968960    6480.000000       ...              7.503300

[5 rows x 8 columns]
BTC-USD_close
BTC-USD_volume
LTC-USD_close
LTC-USD_volume
BCH-USD_close
BCH-USD_volume
ETH-USD_close
ETH-USD_volume


In [3]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict? period= minutes
RATIO_TO_PREDICT = "LTC-USD"

In [4]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

A .shift will just shift the columns for us, a negative shift will shift them "up." So shifting up 3 will give us the price 3 minutes in the future, and we're just assigning this to a new column.

In [5]:
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
print(main_df[[f'{RATIO_TO_PREDICT}_close','future']].head())

            LTC-USD_close     future
time                                
1528968720      96.660004  96.389999
1528968780      96.570000  96.519997
1528968840      96.500000  96.440002
1528968900      96.389999  96.470001
1528968960      96.519997  96.400002


In [6]:
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))
print(main_df[[f'{RATIO_TO_PREDICT}_close','future', 'target']].head(10))

            LTC-USD_close     future  target
time                                        
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
1528969260      96.400002  96.449997       1


Separate out our validation/out of sample data, take the last 5% of the data.

In [7]:
times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

validation_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%


Next, we need to scale and normalize this data. Create our actual sequences and balance the data

In [8]:
from sklearn import preprocessing 
import numpy as np
import random
from collections import deque

def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic. 
  
   # Create our sequences
    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append 

    random.shuffle(sequential_data)  # shuffle for good measure.
    
    # balance the data
    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
        
    return np.array(X), y

In [9]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")


train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930


Making few more constants

In [10]:
import time

EPOCHS = 10  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"  # a unique name for the model

Build the model!

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [19]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [20]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [14]:
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

Model Checkpoint

In [15]:
#filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
#checkpoint = ModelCheckpoint("../input/crypto_data/crypto_data/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

Train the model

In [23]:
# Train model
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    #callbacks=[tensorboard, checkpoint],
)


Train on 77922 samples, validate on 3860 samples
Epoch 1/10
77922/77922 [==============================] - 24s 304us/sample - loss: 0.7244 - acc: 0.5204 - val_loss: 0.6911 - val_acc: 0.5106
Epoch 2/10
77922/77922 [==============================] - 23s 292us/sample - loss: 0.6879 - acc: 0.5447 - val_loss: 0.6819 - val_acc: 0.5645
Epoch 3/10
77922/77922 [==============================] - 23s 291us/sample - loss: 0.6832 - acc: 0.5602 - val_loss: 0.6850 - val_acc: 0.5578
Epoch 4/10
77922/77922 [==============================] - 23s 293us/sample - loss: 0.6815 - acc: 0.5618 - val_loss: 0.6823 - val_acc: 0.5482
Epoch 5/10
77922/77922 [==============================] - 22s 289us/sample - loss: 0.6804 - acc: 0.5666 - val_loss: 0.6768 - val_acc: 0.5661
Epoch 6/10
77922/77922 [==============================] - 22s 289us/sample - loss: 0.6784 - acc: 0.5682 - val_loss: 0.6794 - val_acc: 0.5741
Epoch 7/10
77922/77922 [==============================] - 23s 291us/sample - loss: 0.6768 - acc: 0.5733 -

This is for LTC-USD. Run the same model again for different cryptocurrency and compare 

In [24]:
# Score model
score = model.evaluate(validation_x, validation_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


model.save("RNN_Final")


Test loss: 0.6852892930025881
Test accuracy: 0.5743523
